In [14]:
import pandas as pd
import json
import openai
import requests
from tqdm import tqdm
import ast
import os
import re

In [29]:
with open('openai_key.txt', encoding='utf-8') as f:
    openai_key = f.readlines()[0]
openai.api_key = openai_key

def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

In [179]:
# Read Deconstructed CARDS claims
jc = pd.read_csv('datasets/Deconstructed CARDS claims - Sheet1.csv')
jc

,Code,Claim (short version),Long claim (w/o hidden premise),Fallacy 1,Fallacy 2,Fallacy 3,Fallacy 4,General argument structure,Hidden premise,Denial technique
0,1.1.1.,Antarctica isn't melting,Antarctic sea ice is increasing. Therefore glo...,Cherry picking,Slothful induction,Impossible expectations,Single cause,P1: Antarctica is cooling.\nP2: Sea ice is inc...,P3: Cooling is the only driver of sea ice gain...,P1: Cherry picking\nP2: Slothful induction\nP3...
1,1.1.2.,Greenland isn't melting.,Parts of Greenland's ice sheet are not melting...,Cherry picking,Single cause,Impossible expectations,NaN,P1: Parts of Greenland's ice sheet are not mel...,P2: All parts of ice sheets must be melting un...,"P1: Cherry picking\nP2: Single cause, Impossib..."
2,1.1.3.,Arctic isn’t melting,"In the short term, Arctic sea ice hasn't chang...",Cherry picking,NaN,NaN,NaN,"P1: In the short term, Arctic sea ice hasn't c...",P2: If Arctic sea ice maximum extent hasn't ch...,P2: Cherry picking
3,1.1.4.,Glaciers aren’t melting,Glaciers are growing in some regions. Therefor...,Cherry picking,Oversimplification,NaN,NaN,P1: Glaciers are growing in some regions\nC: G...,P2: If glaciers are growing anywhere in the wo...,"P2: Cherry picking, Oversimplification"
4,1.2,Heading into an ice age,Current climate change is driven by natural fa...,Cherry picking,Slothful induction,Misrepresentation,NaN,P1: Current climate change is driven by natura...,P3: Greenhouse gasses don't have much of a war...,"P1: Cherry picking, Slothful induction\nP3: Mi..."
5,1.3,Weather is cold,Cold weather events are occuring. Therefore gl...,Anecdote,Impossible expectations,NaN,NaN,P1: Cold weather events are occuring.\nC: Glob...,"P2: If global warming was happening, we wouldn...",P1: Anecdote\nP2: Impossible expectations
6,1.4,Hiatus on warming,There's been no warming over a short period. T...,Cherry picking,Slothful induction,Misrepresentation,Cherry picking,P1: There's been no warming over a short perio...,P2: Short time periods are sufficient to make ...,"P1: Cherry picking, Slothful induction\nP2: Mi..."
7,1.6,Sea level rise is exaggerated,Sea levels within a specific time period or re...,Cherry picking,Slothful induction,Oversimplification,NaN,P1: Sea levels within a specific time period o...,P2: Results from a narrow set of sea level ris...,P1: Cherry picking\nP2: Slothful induction\nP3...
8,1.7,Extremes aren't increasing,Extreme weather happened in the past before re...,Single cause,Cherry picking,NaN,NaN,P1: Extreme weather happened in the past befor...,P2: If global warming didn't affect past extre...,P2: Single cause\nP3: Cherry picking
9,2.1.1.,It's the sun,There is a link between solar activity and cli...,Slothful induction,Single cause,NaN,NaN,P1: There is a link between solar activity and...,"P2: If the sun can affect climate, it must be ...",P1: Slothful induction\nP2: Single cause


# Add image 

In [3]:
# Step 1: Deconstruct a claim.
delimiter = "####"
example = "A: Climate has changed naturally in the past, so current climate change is natural.\nP1: Climate has changed naturally in the past.\nP2: Climate is changing now.\nC: Therefore, current climate change is natural."
argument = "Environmentalists get science wrong as they are not committed to scientific principles. Therefore, environmentalists are biased and their reliability is questionable."


system_message = f"""We are going to deconstruct climate misinformation to identify reasoning errors. To do so, we will break down arguments into their starting assumptions or premises and their conclusions.\nFor example, argument A has two premises, P1 and P2, and a conclusion, C:
{example}
Using this structure, please identify the premises and conclusion of the following argument delimited by {delimiter}, reply using JSON format with P# and C as keys:"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{argument}{delimiter}"},  
] 

# response_1 = get_completion_from_messages(messages)
# print(json.dumps(response_1))

In [183]:
# Get ChatGPT response for each claim in jc dataset.

for i in jc.index:
    argument = jc.loc[i, 'Long claim (w/o hidden premise)']
    messages =  [{'role':'system', 'content': system_message},    
                 {'role':'user', 'content': f"{delimiter}{argument}{delimiter}"}]
    response_1 = get_completion_from_messages(messages)
    with open(f'structure_{i}.json', 'w') as f:
        json.dump(json.dumps(response_1), f)
    

In [217]:
# Add original claim to each response and store dict as json

for i in os.listdir('outputs3/'):
    print(i)
    idx = int(i.split('.')[0].split('_')[1])
    with open(f'outputs3/{i}', 'r') as f:
        response = json.load(f)
        response = json.loads(response)
        response = ast.literal_eval(response)
        response['A'] = jc.loc[idx, 'Long claim (w/o hidden premise)']

    with open(f'outputs3/structure_{idx}.json', 'w') as f:
        json.dump(response, f)

structure_16.json
structure_41.json
structure_3.json
structure_20.json
structure_36.json
structure_37.json
structure_21.json
structure_2.json
structure_40.json
structure_17.json
structure_26.json
structure_30.json
structure_9.json
structure_10.json
structure_5.json
structure_4.json
structure_11.json
structure_8.json
structure_31.json
structure_27.json
structure_32.json
structure_24.json
structure_7.json
structure_45.json
structure_12.json
structure_28.json
structure_29.json
structure_13.json
structure_44.json
structure_6.json
structure_25.json
structure_33.json
structure_1.json
structure_43.json
structure_14.json
structure_38.json
structure_18.json
structure_34.json
structure_22.json
structure_23.json
structure_35.json
structure_19.json
structure_39.json
structure_15.json
structure_42.json
structure_0.json


In [112]:
# Step 2: Check Validity.

example = f"""
Logically valid:
A: 31,000 dissenting scientists prove there is no expert agreement on human-caused global warming"
P1: A large proportion of people with science degrees dissent human-caused global warming.
P2: People with science degreess are experts on climante change.
C: There is no expert agreement on human-caused global warming.

TRUE, This argument is logically valid, if it was true that a large proportion of science graduates dissented and all those science graduates were climate experts then yes, it would follow that there was no expert consensus on climate change.

Logically invalid: 
A: Climate has changed naturally in the past, so current climate change is natural.
P1: Climate has changed naturally in the past.
P2: Climate is changing now.
C: Therefore, current climate change is natural.

FALSE, This argument is logically invalid, just because the climate changed naturally in the past doesn't mean it's changing naturally now. """

example_parallel_argument = f"""
Argument: Climate has changed naturally in the past, so current climate change is natural.
Parallel argument: Because people died of cancer in the past, cigarettes can't be the cause of any cancer now.

Apply paralell argument to show how ridiculous the argument really is, for example:
Myth:
P1: it's cold
C: Global warming doesn't exist

Parallel argument:
P1: It's dark
C: The sun doesn't exist"""

system_message = f"""Check if the argument delimited by {delimiter} is logically valid i.e. Does the conclusion follow from the premises?
Assume that if all the premises are true, does it follow the conclusion is also true.
Read the following examples before giving your answer, A: Argument, P#: Premises and C: Conclusion
Examples: {example}

reply using JSON format in this structure:
"valid" : <TRUE or FALSE>,
"reason": <Explain your reasoning>,
"""

# Is usually simpler to demonstrate an argument is invalid by use of a parallel argument. Parallel arguments use the same logical structure as the target argument but instantiate it in such a way that the conclusion is obviously false. 
# In a valid argument it is impossible for the conclusion to be false whilst all the premises are true. 
# Parallel arguments identify a clear counter example and show that an argument is not valid. If the conclusion is not justified by the reasons given, so we must either reject the conclusion or revise it.
# {example_parallel_argument}
# "parallel_argument": <Give your parallel argument>


messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}A:{argument}{'response'}{delimiter}"}, 
] 

# response_2 = get_completion_from_messages(messages)
# print(json.dumps(response_2))

In [113]:
[os.listdir('outputs3/')[35]]

['structure_38.json']

In [114]:
for i in [os.listdir('outputs3/')[35]]:

    with open(f'outputs3/{i}', 'r') as f:
        response = json.load(f)
        keys = list(response.keys())
        p_keys = sorted([k for k in keys if k.startswith('P')], key=lambda x: int(re.sub("\D", "", x)))

        argument = []
        argument.append(f"A: {response['A']}")
        for key in p_keys:
            argument.append(f"{key}: {response[key]}")
        argument.append(f"C: {response['C']}")

        argument = "\n".join(argument)
        messages =  [  
                    {'role':'system', 
                    'content': system_message},    
                    {'role':'user', 
                    'content': f"{delimiter}{argument}{delimiter}"}, 
                    ] 

        response_2 = get_completion_from_messages(messages)
        try:
            json_response = json.loads(response_2)
            response['valid'] = json_response['valid']
            response['reason'] = json_response['reason']
        except json.JSONDecodeError:
            valid_str, reason_str = response_2.split(',', 1)
            response['valid'] = valid_str.strip()
            response['reason'] = reason_str.strip()
        except:
            print(f'{i}:{response_2}')

        try:
            with open(f'outputs3/{i}', 'w') as f:
                json.dump(response, f)
        except:
            print(f'Error on: {i}')

In [115]:
# Step 2a: identify hidden premises.

example = "A: Climate has changed naturally in the past, so current climate change is natural.\nP1: Climate has changed naturally in the past.\nP2: Climate is changing now.\nHP: If something wasn't a cause in the past, it can't be a cause now\nC: Therefore, current climate change is natural."

system_message = f"""The conclusion of the argument delimited by {delimiter} doesn't follow from their premises, i.e. is logically invalid.
We need to add an extra premise, an unstated assumption to make this argument logically valid. Follow the example bellow, A: Argument, P#: Premises, HP: Hidden premise C: Conclusion.
Example: 
{example} 

reply using JSON format in this structure:
{{"HP": "<Hidden premise>"}}

"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{argument}{delimiter}"}, 
] 

# "motive": <Reason why this hidden premise makes the argument valid>
# response_3 = get_completion_from_messages(messages)
# print(json.dumps(response_3))


In [116]:
print(system_message)
print(f"{delimiter}{argument}{delimiter}")

The conclusion of the argument delimited by #### doesn't follow from their premises, i.e. is logically invalid.
We need to add an extra premise, an unstated assumption to make this argument logically valid. Follow the example bellow, A: Argument, P#: Premises, HP: Hidden premise C: Conclusion.
Example: 
A: Climate has changed naturally in the past, so current climate change is natural.
P1: Climate has changed naturally in the past.
P2: Climate is changing now.
HP: If something wasn't a cause in the past, it can't be a cause now
C: Therefore, current climate change is natural. 

reply using JSON format in this structure:
{"HP": "<Hidden premise>"}


####A: Climate model predictions don't match observations. Therefore climate models are unreliable.
P1: Climate model predictions don't match observations.
C: Therefore, climate models are unreliable.####


In [117]:
for i in os.listdir('outputs3/')[35:]:

    with open(f'outputs3/{i}', 'r') as f:
        print(i)
        response = json.load(f)
        if response['valid'] == 'FALSE':
            keys = list(response.keys())
            p_keys = sorted([k for k in keys if k.startswith('P')], key=lambda x: int(re.sub("\D", "", x)))

            argument = []
            argument.append(f"A: {response['A']}")
            for key in p_keys:
                argument.append(f"{key}: {response[key]}")
            argument.append(f"C: {response['C']}")

            argument = "\n".join(argument)
            messages =  [  
                        {'role':'system', 
                        'content': system_message},    
                        {'role':'user', 
                        'content': f"{delimiter}{argument}{delimiter}"}, 
                        ] 

            response_3 = get_completion_from_messages(messages)
            try:
                json_response = json.loads(response_3)
                response['HP'] = json_response['HP']
            except:
                print(f'{i}:{response_3}')

            try:
                with open(f'outputs3/{i}', 'w') as f:
                    json.dump(response, f)
            except:
                print(f'Error on: {i}')

structure_38.json
structure_18.json
structure_34.json
structure_22.json
structure_23.json
structure_35.json
structure_19.json
structure_39.json
structure_15.json
structure_42.json
structure_0.json


| TECHNIQUE | DEFINITION | EXAMPLE |
|---|---|---|
| Ad Hominem | Attacking a person/group instead of addressing their arguments. | “Climate science can’t be trusted because climate scientists are biased.” |
| Ambiguity | Using ambiguous language in order to lead to a misleading conclusion. | “Thermometer readings have uncertainty which means we don’t know whether global warming is happening.” |
| Anecdote | Using personal experience or isolated examples instead of sound arguments or compelling evidence. | “The weather is cold today—whatever happened to global warming?” |
| Blowfish | Focusing on an inconsequential aspect of scientific research, blowing it out of proportion in order to distract from or cast doubt on the main conclusions of the research. | “The hockey stick graph is invalid because it contains statistical errors.” |
| Bulk Fake Experts | Citing large numbers of seeming experts to argue that there is no scientific consensus on a topic. | “There is no expert consensus because 31,487 Americans with a science degree signed a petition saying humans aren’t disrupting climate.” |
| Cherry Picking | Carefully selecting data that appear to confirm one position while ignoring other data that contradicts that position. | “Global warming stopped in 1998.” |
| Contradictory | Simultaneously believing in ideas that are mutually contradictory.  | “The temperature record is fabricated by scientists… the temperature record shows cooling.” |
| Conspiracy Theory | Proposing that a secret plan exists to implement a nefarious scheme such as hiding a truth. | “The climategate emails prove that climate scientists have engaged in a conspiracy to deceive the public.” |
| Fake Debate | Presenting science and pseudoscience in an adversarial format to give the false impression of an ongoing scientific debate. | “Climate deniers should get equal coverage with climate scientists, providing a more balanced presentation of views.” |
| Fake Experts | Presenting an unqualified person or institution as a source of credible information. | “A retired physicist argues against the climate consensus, claiming the current weather change is just a natural occurrence.” |
| False Analogy | Assuming that because two things are alike in some ways, they are alike in some other respect. | “Climate skeptics are like Galileo who overturned the scientific consensus about geocentrism.” |
| False Choice | Presenting two options as the only possibilities, when other possibilities exist. | “CO2 lags temperature in the ice core record, proving that temperature drives CO2, not the other way around.” |
| False Equivalence<br>(apples vs. oranges) | Incorrectly claiming that two things are equivalent, despite the fact that there are notable differences between them. | “Why all the fuss about COVID when thousands die from the flu every year.” |
| Immune to evidence | Re-interpreting any evidence that counters a conspiracy theory as originating from the conspiracy. | “Those investigations finding climate scientists aren’t conspiring were part of the conspiracy.” |
| Impossible Expectations | Demanding unrealistic standards of certainty before acting on the science. | “Scientists can’t even predict the weather next week. How can they predict the climate in 100 years?” |
| Logical Fallacies | Arguments where the conclusion doesn’t logically follow from the premises. Also known as a non sequitur. | “Climate has changed naturally in the past so what’s happening now must be natural.” |
| Magnified Minority | Magnifying the significance of a handful of dissenting scientists to cast doubt on an overwhelming scientific consensus. | “Sure, there’s 97% consensus but Professor Smith disagrees with the consensus position.” |
| Misrepresentation | Misrepresenting a situation or an opponent’s position in such a way as to distort understanding. | “They changed the name from ‘global warming’ to ‘climate change’ because global warming stopped happening.” |
| Moving Goalposts | Demanding higher levels of evidence after receiving requested evidence. | “Sea levels may be rising but they’re not accelerating.” |
| Nefarious intent | Assuming that the motivations behind any presumed conspiracy are nefarious. | “Climate scientists promote the climate hoax because they’re in it for the money.” |
| Overriding suspicion  | Having a nihilistic degree of scepticism towards the official account, preventing belief in anything that doesn’t fit into the conspiracy theory.  | “Show me one line of evidence for climate change… oh, that evidence is faked!” |
| Oversimplification | Simplifying a situation in such a way as to distort understanding, leading to erroneous conclusions. | “CO2 is plant food so burning fossil fuels will be good for plants.” |
| Persecuted victim | Perceiving and presenting themselves as the victim of organised persecution. | “Climate scientists are trying to take away our freedom.” |
| Quote Mining | Taking a person’s words out-of-context in order to misrepresent their position. | “Mike’s trick… to hide the decline.” |
| Re-interpreting randomness | Believing that nothing occurs by accident, so that random events are re-interpreted as being caused by the conspiracy. | “NASA’s satellite exploded? They must be trying to hide inconvenient data!” |
| Red Herring | Deliberately diverting attention to an irrelevant point to distract from a more important point. | “CO2 is a trace gas so it’s warming effect is minimal.” |
| Single Cause | Assuming a single cause or reason when there might be multiple causes or reasons. | “Climate has changed naturally in the past so what’s happening now must be natural.” |
| Slippery Slope | Suggesting that taking a minor action will inevitably lead to major consequences. | “If we implement even a modest climate policy, it will start us down the slippery slope to socialism and taking away our freedom.” |
| Slothful Induction | Ignoring relevant evidence when coming to a conclusion. | “There is no empirical evidence that humans are causing global warming.” |
| Something must be wrong | Maintaining that “something must be wrong” and the official account is based on deception, even when specific parts of a conspiracy theory become untenable. | “Ok, fine, 97% of climate scientists agree that humans are causing global warming, but that’s just because they’re toeing the party line.” |
| Straw Man | Misrepresenting or exaggerating an opponent’s position to make it easier to attack. | “In the 1970s, climate scientists were predicting an ice age. |

In [122]:
# Step3: Check premises

system_message = f"""Evaluate if the premise delimited by {delimiter} is logically valid or if it corresponds to a fallacy from the following table.
It is mandatory to analyse each fallacy from the table below and provide a comprehensive analysis for each one of them using JSONL format. Ensure to include both "yes" and "no" answers for each analysis. The format should be as follows:
{{"technique": "<technique name>", "yes/no": "<YES>", "explanation": "<explanation for YES>"}}
{{"technique": "<technique name>", "yes/no": "<NO>", "explanation": "<explanation for NO>"}}
{{"technique": "<technique name>", "yes/no": "<YES>", "explanation": "<explanation for YES>"}}
{{"technique": "<technique name>", "yes/no": "<NO>", "explanation": "<explanation for NO>"}}

| TECHNIQUE | DEFINITION | EXAMPLE |
|---|---|---|
| Ad Hominem | Attacking a person/group instead of addressing their arguments. | “Climate science can’t be trusted because climate scientists are biased.” |
| Ambiguity | Using ambiguous language in order to lead to a misleading conclusion. | “Thermometer readings have uncertainty which means we don’t know whether global warming is happening.” |
| Anecdote | Using personal experience or isolated examples instead of sound arguments or compelling evidence. | “The weather is cold today—whatever happened to global warming?” |
| Blowfish | Focusing on an inconsequential aspect of scientific research, blowing it out of proportion in order to distract from or cast doubt on the main conclusions of the research. | “The hockey stick graph is invalid because it contains statistical errors.” |
| Bulk Fake Experts | Citing large numbers of seeming experts to argue that there is no scientific consensus on a topic. | “There is no expert consensus because 31,487 Americans with a science degree signed a petition saying humans aren’t disrupting climate.” |
| Cherry Picking | Carefully selecting data that appear to confirm one position while ignoring other data that contradicts that position. | “Global warming stopped in 1998.” |
| Contradictory | Simultaneously believing in ideas that are mutually contradictory.  | “The temperature record is fabricated by scientists… the temperature record shows cooling.” |
| Conspiracy Theory | Proposing that a secret plan exists to implement a nefarious scheme such as hiding a truth. | “The climategate emails prove that climate scientists have engaged in a conspiracy to deceive the public.” |
| Fake Debate | Presenting science and pseudoscience in an adversarial format to give the false impression of an ongoing scientific debate. | “Climate deniers should get equal coverage with climate scientists, providing a more balanced presentation of views.” |
| Fake Experts | Presenting an unqualified person or institution as a source of credible information. | “A retired physicist argues against the climate consensus, claiming the current weather change is just a natural occurrence.” |
| False Analogy | Assuming that because two things are alike in some ways, they are alike in some other respect. | “Climate skeptics are like Galileo who overturned the scientific consensus about geocentrism.” |
| False Choice | Presenting two options as the only possibilities, when other possibilities exist. | “CO2 lags temperature in the ice core record, proving that temperature drives CO2, not the other way around.” |
| False Equivalence<br>(apples vs. oranges) | Incorrectly claiming that two things are equivalent, despite the fact that there are notable differences between them. | “Why all the fuss about COVID when thousands die from the flu every year.” |
| Immune to evidence | Re-interpreting any evidence that counters a conspiracy theory as originating from the conspiracy. | “Those investigations finding climate scientists aren’t conspiring were part of the conspiracy.” |
| Impossible Expectations | Demanding unrealistic standards of certainty before acting on the science. | “Scientists can’t even predict the weather next week. How can they predict the climate in 100 years?” |
| Logical Fallacies | Arguments where the conclusion doesn’t logically follow from the premises. Also known as a non sequitur. | “Climate has changed naturally in the past so what’s happening now must be natural.” |
| Magnified Minority | Magnifying the significance of a handful of dissenting scientists to cast doubt on an overwhelming scientific consensus. | “Sure, there’s 97% consensus but Professor Smith disagrees with the consensus position.” |
| Misrepresentation | Misrepresenting a situation or an opponent’s position in such a way as to distort understanding. | “They changed the name from ‘global warming’ to ‘climate change’ because global warming stopped happening.” |
| Moving Goalposts | Demanding higher levels of evidence after receiving requested evidence. | “Sea levels may be rising but they’re not accelerating.” |
| Nefarious intent | Assuming that the motivations behind any presumed conspiracy are nefarious. | “Climate scientists promote the climate hoax because they’re in it for the money.” |
| Overriding suspicion  | Having a nihilistic degree of scepticism towards the official account, preventing belief in anything that doesn’t fit into the conspiracy theory.  | “Show me one line of evidence for climate change… oh, that evidence is faked!” |
| Oversimplification | Simplifying a situation in such a way as to distort understanding, leading to erroneous conclusions. | “CO2 is plant food so burning fossil fuels will be good for plants.” |
| Persecuted victim | Perceiving and presenting themselves as the victim of organised persecution. | “Climate scientists are trying to take away our freedom.” |
| Quote Mining | Taking a person’s words out-of-context in order to misrepresent their position. | “Mike’s trick… to hide the decline.” |
| Re-interpreting randomness | Believing that nothing occurs by accident, so that random events are re-interpreted as being caused by the conspiracy. | “NASA’s satellite exploded? They must be trying to hide inconvenient data!” |
| Red Herring | Deliberately diverting attention to an irrelevant point to distract from a more important point. | “CO2 is a trace gas so it’s warming effect is minimal.” |
| Single Cause | Assuming a single cause or reason when there might be multiple causes or reasons. | “Climate has changed naturally in the past so what’s happening now must be natural.” |
| Slippery Slope | Suggesting that taking a minor action will inevitably lead to major consequences. | “If we implement even a modest climate policy, it will start us down the slippery slope to socialism and taking away our freedom.” |
| Slothful Induction | Ignoring relevant evidence when coming to a conclusion. | “There is no empirical evidence that humans are causing global warming.” |
| Something must be wrong | Maintaining that “something must be wrong” and the official account is based on deception, even when specific parts of a conspiracy theory become untenable. | “Ok, fine, 97% of climate scientists agree that humans are causing global warming, but that’s just because they’re toeing the party line.” |
| Straw Man | Misrepresenting or exaggerating an opponent’s position to make it easier to attack. | “In the 1970s, climate scientists were predicting an ice age. |

"""

# messages =  [  
# {'role':'system', 
#  'content': system_message},    
# {'role':'user', 
#  'content': f"{delimiter}{json.loads(response_1).get('P1')}{delimiter}"}, 
# ]

# response_4 = get_completion_from_messages(messages)
# print(json.dumps(response_4))

In [166]:
# json.loads(response_4)

{'technique': 'Ad Hominem',
 'yes/no': 'YES',
 'explanation': 'The statement attacks environmentalists instead of addressing their arguments or evidence.'}

In [123]:
# print(f"{delimiter}{json.loads(response_1).get('P1')}{delimiter}")

In [157]:
# for i, f in enumerate([os.listdir('outputs3/')[10]]):
for i, f in enumerate(['structure_4.json', 'structure_8.json', 'structure_31.json', 'structure_24.json', 'structure_29.json', 'structure_1.json', 'structure_23.json', 'structure_15.json']):

    with open(f'outputs3/{f}', 'r') as fl:
        print(i, f)
        response = json.load(fl)
        keys = list(response.keys())
        p_keys = sorted([k for k in keys if k.startswith('P')], key=lambda x: int(re.sub("\D", "", x)))
        for key in p_keys:
            messages =  [  
            {'role':'system', 
            'content': system_message},    
            {'role':'user', 
            'content': f"{delimiter}{response[key]}{delimiter}"}, 
            ] 

            response_4 = get_completion_from_messages(messages)
            try:
                response[key] = {'premise': response[key]}
                for k,v in json.loads(response_4).items():
                    response[key][k] = v

                with open(f'outputs3/{f}', 'w') as fl:
                    json.dump(response, fl)
            except Exception as e:
                print(f'Error on {i} {f} {key}: {e}')
                print(response_4)

        if 'HP' in response:
            messages =  [  
            {'role':'system', 
            'content': system_message},    
            {'role':'user', 
            'content': f"{delimiter}{response['HP']}{delimiter}"}, 
            ] 

            response_4 = get_completion_from_messages(messages)
            try:
                response['HP'] = {'premise': response['HP']}
                for k,v in json.loads(response_4).items():
                    response['HP'][k] = v
                with open(f'outputs3/{f}', 'w') as fl:
                    json.dump(response, fl)
            except Exception as e:
                print(f'Error on {i} {f} {key}: {e}')
                print(response_4)


0 structure_4.json
Error on 0 structure_4.json P2: Extra data: line 2 column 1 (char 323)
{"technique": "Single Cause", "yes/no": "YES", "explanation": "The premise suggests that there is only one cause of climate change, which is natural factors. This is a fallacy because there are multiple factors that contribute to climate change, including human activities such as burning fossil fuels and deforestation."}
{"technique": "False Equivalence (apples vs. oranges)", "yes/no": "NO", "explanation": "There is no false equivalence in the premise."}
{"technique": "Logical Fallacies", "yes/no": "YES", "explanation": "The premise commits the logical fallacy of oversimplification by reducing the complex issue of climate change to a single cause. It also commits the fallacy of false dilemma by presenting only two options (natural factors or human activities) when there are actually multiple factors that contribute to climate change."}
1 structure_8.json
Error on 1 structure_8.json P1: Extra data:

In [163]:
import json

# Load the data from the JSON file
with open('outputs3/structure_23.json') as f:
    data = json.load(f)

# Function to create markdown table
def create_markdown_table(data):
    # Header
    table = "| N | Premises | Denial Technique | Explanation |\n"
    table += "|---|---|---|---|\n"

    # Iterate over the keys in data to fill in the table rows
    for key in ['P1', 'P2', 'HP']:  # ensure the order
        if key in data:
            premises = data[key]['premise']
            if 'techniques' in data[key]:
                if isinstance(data[key]['techniques'], list):
                    denial_technique = " ".join([f"{technique['technique']} | {technique['explanation']}" for technique in data[key]['techniques']])
                else:
                    denial_technique = f"{data[key]['techniques']['technique']} | {data[key]['techniques']['explanation']}"
            elif 'technique' in data[key]:
                denial_technique = f"{data[key]['technique']} | {data[key]['explanation']}"
            else:
                denial_technique = 'No technique data available'
            table += f"| {key} | {premises} | {denial_technique} |\n"

    return table

# Display the markdown table
from IPython.display import display, Markdown

print(data)

display(Markdown(create_markdown_table(data)))


{'P1': {'premise': 'Climate policy increases energy costs', 'techniques': [{'technique': 'False Choice', 'yes/no': 'YES', 'explanation': 'The premise presents a false choice between implementing climate policy and keeping energy costs low, when in reality there are ways to address climate change without significantly increasing energy costs. Additionally, the premise assumes that the only cost of climate change is the cost of implementing policy, ignoring the potentially catastrophic costs of inaction.'}, {'technique': 'Oversimplification', 'yes/no': 'YES', 'explanation': 'The premise oversimplifies the relationship between climate policy and energy costs, ignoring the complexity of the issue and the potential for innovative solutions that could mitigate the impact of policy on energy costs.'}]}, 'P2': {'premise': 'Increased energy costs will have harmful effects', 'technique': 'Logical Fallacies', 'yes/no': 'YES', 'explanation': "The premise is a non sequitur, as it doesn't logically 

| N | Premises | Denial Technique | Explanation |
|---|---|---|---|
| P1 | Climate policy increases energy costs | False Choice | The premise presents a false choice between implementing climate policy and keeping energy costs low, when in reality there are ways to address climate change without significantly increasing energy costs. Additionally, the premise assumes that the only cost of climate change is the cost of implementing policy, ignoring the potentially catastrophic costs of inaction. Oversimplification | The premise oversimplifies the relationship between climate policy and energy costs, ignoring the complexity of the issue and the potential for innovative solutions that could mitigate the impact of policy on energy costs. |
| P2 | Increased energy costs will have harmful effects | Logical Fallacies | The premise is a non sequitur, as it doesn't logically follow that increased energy costs will have harmful effects. While it is possible that increased energy costs could have harmful effects, it is not necessarily true in all cases. |
| HP | Assuming that the harmful effects of increased energy costs outweigh the benefits of implementing climate policy. | False Dilemma | The premise presents a false dilemma by assuming that there are only two options: either implement climate policy or avoid increased energy costs. This ignores the possibility that climate policy could have benefits that outweigh the costs, or that there may be other ways to address climate change that do not involve increased energy costs. |


In [171]:
import json
import csv

with open('outputs3/structure_23.json') as json_file:
    data = json.load(json_file)

with open('output.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Write header
    writer.writerow(["N", "Premises", "Denial Technique", "Explanation"])

    for key in ['P1', 'P2', 'HP']:
        if key in data:
            premise = data[key]['premise']
            if 'techniques' in data[key]:
                techniques = "\n".join([f"{t['technique']} : {t['explanation']}" for t in data[key]['techniques']])
            else:
                techniques = f"{data[key]['technique']} : {data[key]['explanation']}"

            # Write row
            writer.writerow([key, premise, techniques, ""])


In [167]:
from IPython.display import display, HTML

# Assuming that data is the JSON data

html_table = create_html_table(data)

display(HTML(html_table))

In [168]:
html_table

"<table><tr><th>N</th><th>Premises</th><th>Denial Technique</th><th>Explanation</th></tr><tr><td>P1</td><td>Climate policy increases energy costs</td><td>False Choice : The premise presents a false choice between implementing climate policy and keeping energy costs low, when in reality there are ways to address climate change without significantly increasing energy costs. Additionally, the premise assumes that the only cost of climate change is the cost of implementing policy, ignoring the potentially catastrophic costs of inaction.<br>Oversimplification : The premise oversimplifies the relationship between climate policy and energy costs, ignoring the complexity of the issue and the potential for innovative solutions that could mitigate the impact of policy on energy costs.</td></tr><tr><td>P2</td><td>Increased energy costs will have harmful effects</td><td>Logical Fallacies : The premise is a non sequitur, as it doesn't logically follow that increased energy costs will have harmful e